In [1]:
include("../LiPoSID.jl")

include("CtrlSID.jl")

using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

using HDF5
using DynamicPolynomials

using Dates

using Statistics

┌ Warning: Package LinearSolve does not have KrylovKit in its dependencies:
│ - If you have LinearSolve checked out for development and have
│   added KrylovKit as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LinearSolve
└ Loading KrylovKit into LinearSolve from project dependency, future warnings for LinearSolve are suppressed.


In [14]:
function det_2x2(matrix)
    # Ensure the matrix is 2x2
    if size(matrix) != (2, 2)
        throw(ArgumentError("Matrix must be 2x2"))
    end

    # Calculate the determinant
    return matrix[1, 1] * matrix[2, 2] - matrix[1, 2] * matrix[2, 1]
end

function fidelity_norm(ρ, σ)
    tr(ρ * σ)+2*sqrt(det_2x2(ρ)*det_2x2(σ))
end

function pseudo_fidelity_norm(ρ, σ)
    tr(ρ * σ)+(det_2x2(ρ)*det_2x2(σ))^2
end

pseudo_fidelity_norm (generic function with 1 method)

In [2]:
σˣ = [ 0 1 
       1 0 ]

σʸ = [ 0.   im*1
      -im*1 0    ]

σᶻ = [ 1.  0
       0  -1 ] 

fᴷ₁ = σˣ/2
fᴷ₂ = σʸ/2
fᴷ₃ = σᶻ/2

@assert tr(σˣ/2*σʸ/2) == tr(σˣ/2*σᶻ/2) ==  tr(σʸ/2*σᶻ/2) ≈ 0
@assert tr(σˣ/2*σˣ/2) == tr(σʸ/2*σʸ/2) == tr(σᶻ/2*σᶻ/2) ≈ 1/2

fᴷᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

@polyvar m_Re[1:3, 1:3] m_Im[1:3, 1:3]

M = m_Re + im * m_Im
Cˢʸᵐᵇ = M * (M')

@polyvar ϵ h_Re h_Im # h₁ h₂ h₃

H0ˢʸᵐᵇ = [ ϵ               h_Re+im*h_Im
           h_Re-im*h_Im   -ϵ            ] / 2

H0ᴷˢʸᵐᵇ = h_Re * fᴷ₁ + h_Im * fᴷ₂  + ϵ * fᴷ₃ 

@assert tr(H0ᴷˢʸᵐᵇ) == 0
@assert H0ᴷˢʸᵐᵇ == H0ˢʸᵐᵇ

Hˢʸᵐᵇ = H0ˢʸᵐᵇ

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (0.5+0.0im)ϵ                       (0.5+0.0im)h_Re + (0.0+0.5im)h_Im
 (0.5+0.0im)h_Re + (0.0-0.5im)h_Im  (-0.5+0.0im)ϵ

In [11]:
using StaticArrays

function kossak_obj(ρ, t, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ)

    function Dc(ρ, t)
        U = (H0ˢʸᵐᵇ*ρ - ρ*H0ˢʸᵐᵇ)/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    obj = 0
    for i in 3:length(ρ)
        obj += LiPoSID.frobenius_norm2(
            ρ[i] - ρ[i-2] - (t[i]-t[i-1])*(Dc(ρ[i], t[i])+
            4*Dc(ρ[i-1], t[i-1])+Dc(ρ[i-2], t[i-2]))/3
        )
    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    return obj

end


function kossak_obj_pseudo_fidelity(ρ, t, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, Fᴼᴺᴮ)

    function Dc(ρ, t)
        U = (H0ˢʸᵐᵇ*ρ - ρ*H0ˢʸᵐᵇ)/im 
        D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in Fᴼᴺᴮ, fⱼ in Fᴼᴺᴮ])/2
        return U + D
    end 

    obj = 0
    for i in 3:length(ρ)
        obj += pseudo_fidelity_norm(
            ρ[i],
            ρ[i-2] - (t[i]-t[i-1])*(Dc(ρ[i], t[i])+
            4*Dc(ρ[i-1], t[i-1])+Dc(ρ[i-2], t[i-2]))/3
        )
    end

    if isempty(monomials(obj))
        obj = 0. 
    else
        obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    end

    return obj

end

kossak_obj_pseudo_fidelity (generic function with 1 method)

In [12]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

function read_GEXY_timeevolution(file_name, γ)

    tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
    tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
    tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)
    tʸ, ρʸ = read_timeevolution(file_name, "B4", γ)

    ρᵍᵉˣʸ = ρᵍ, ρᵉ, ρˣ, ρʸ 
    tᵍᵉˣʸ = tᵍ, tᵉ, tˣ, tʸ

    return tᵍᵉˣʸ , ρᵍᵉˣʸ 

end

function kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_obj(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyE = kossak_obj(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyX = kossak_obj(ρˣ, tˣ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyY = kossak_obj(ρʸ, tʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end


function kossak_GEXY_obj_pseudo_fidelity(ρᵍᵉˣʸ, tᵍᵉˣʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_obj_pseudo_fidelity(ρᵍ, tᵍ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyE = kossak_obj_pseudo_fidelity(ρᵉ, tᵉ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyX = kossak_obj_pseudo_fidelity(ρˣ, tˣ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)
    polyY = kossak_obj_pseudo_fidelity(ρʸ, tʸ, H0ˢʸᵐᵇ, Cˢʸᵐᵇ, fᴼᴺᴮ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end

kossak_GEXY_obj_pseudo_fidelity (generic function with 1 method)

In [15]:
evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

#γᵢ = "0.079477"

γᵢ = "0.25133"

tᵍᵉˣʸ , ρᵍᵉˣʸ  = read_GEXY_timeevolution(evol_data_file_name, γᵢ)
    
#polyGEXY = kossak_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

polyGEXY = kossak_GEXY_obj_pseudo_fidelity(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Cˢʸᵐᵇ, fᴷᴼᴺᴮ)

@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))

minimum(abs.(coefficients(polyGEXY))) = 1.1869459682199748e-66
maximum(abs.(coefficients(polyGEXY))) = 13324.675418158668


13324.675418158668

minimum(abs.(coefficients(polyGEXY))) = 4.658513032068184e-37
maximum(abs.(coefficients(polyGEXY))) = 65.37495321955478


65.37495321955478

In [9]:
length(variables(polyGEXY))

21

In [51]:
using BlackBoxOptim

initial_guess = [1.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,25.,0.,0];

res = compare_optimizers(polyGEXY; SearchRange = (-260, 260),
 NumDimensions =length(variables(polyGEXY)), 
 MaxTime = 25.0,
 InitialPopulation = [initial_guess]);

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, SimpleSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.50 secs, 4278 evals, 4146 steps, improv/step: 0.177 (last = 0.1768), fitness=7443.092601836
1.00 secs, 10534 evals, 10402 steps, improv/step: 0.153 (last = 0.1371), fitness=173.274098181
1.50 secs, 16529 evals, 16397 steps, improv/step: 0.144 (last = 0.1293), fitness=1.443947708
2.00 secs, 23262 evals, 23130 steps, improv/step: 0.135 (last = 0.1109), fitness=0.030815471
2.50 secs, 29802 evals, 29670 steps, improv/step: 0.126 (last = 0.0951), fitness=0.001885729
3.00 secs, 35962 evals, 35830 steps, improv/step: 0.113 (last = 0.0521), fitness=0.001230314
3.50 secs, 42522 evals, 42390 steps, improv/step: 0.104 (last = 0.0526), fitness=0.000698769
4.00 secs, 48599 evals, 48467 steps, improv/step: 0.096 (last = 0.0428), fitness=0.000581050
4.50 secs, 55275 evals, 55143 steps, improv/step: 0.089 

In [35]:
using TSSOS
sol_sos, opt_val_sos, status_sos = CtrlSID.tssos(polyGEXY)

(PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im] => [1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10  …  1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 1.0e-10, 25.126102787127508, 8.87543986877677e-5, -7.613692874867376e-5], 0.0005504467486778955, :Local)

In [66]:
initial_guess = [0.127,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,25.126,0.,0];

init_solution = variables(polyGEXY) => initial_guess

subs(polyGEXY, init_solution)

0.011842594071879375

In [60]:
subs(polyGEXY, init_solution)

0.02449128201470785

In [ ]:
subs(polyGEXY, sol_tssos)

In [ ]:
Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol); Cˢⁱᵈ = subs(Cˢʸᵐᵇ, sol)
epsilon = subs(ϵ, sol)

In [50]:
result_bbo = bboptimize(polyGEXY; SearchRange = (-26.0, 26.0), NumDimensions = 21,
    MaxTime = 20, 
    Method = :adaptive_de_rand_1_bin_radiuslimited,
    InitialPopulation = [initial_guess])

optimal_value_bbo = best_candidate(result_bbo)
minimum_function_value_bbo = best_fitness(result_bbo)
    

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, RadiusLimitedSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
0.50 secs, 4658 evals, 4568 steps, improv/step: 0.196 (last = 0.1964), fitness=3642.209623575
1.00 secs, 10644 evals, 10556 steps, improv/step: 0.184 (last = 0.1747), fitness=10.810173457
1.50 secs, 17200 evals, 17112 steps, improv/step: 0.179 (last = 0.1698), fitness=0.012684721
2.00 secs, 23340 evals, 23253 steps, improv/step: 0.163 (last = 0.1179), fitness=0.000677234
2.50 secs, 30004 evals, 29918 steps, improv/step: 0.155 (last = 0.1304), fitness=0.000566960
3.06 secs, 36452 evals, 36369 steps, improv/step: 0.152 (last = 0.1370), fitness=0.000552486
3.56 secs, 43199 evals, 43117 steps, improv/step: 0.148 (last = 0.1233), fitness=0.000550754
4.06 secs, 49718 evals, 49638 steps, improv/step: 0.143 (last = 0.1115), fitness=0.000550464
4.56 secs, 55909 evals, 55829 steps, improv/step: 

0.000550440538163369

In [41]:
sol = variables(polyGEXY) => optimal_value_bbo

PolyVar{true}[m_Re₁₋₁, m_Re₂₋₁, m_Re₃₋₁, m_Re₁₋₂, m_Re₂₋₂, m_Re₃₋₂, m_Re₁₋₃, m_Re₂₋₃, m_Re₃₋₃, m_Im₁₋₁  …  m_Im₃₋₁, m_Im₁₋₂, m_Im₂₋₂, m_Im₃₋₂, m_Im₁₋₃, m_Im₂₋₃, m_Im₃₋₃, ϵ, h_Re, h_Im] => [-0.019252360213749303, 0.09690703029401142, 1.8258582847707846e-6, -0.12149919235510943, 0.0955439117611729, 2.190662827733508e-5, -0.05860726215875083, -0.159344149788373, 1.541100858578782e-5, 0.15264688254369485  …  -3.2032859934229064e-5, 0.14734242070961437, 0.07396435694379139, -3.181074862550541e-5, -0.25377789453532956, 0.0420059867043054, 5.168442308255742e-5, 25.126102756390313, 8.880404673250727e-5, -7.609725815641716e-5]

In [42]:
Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol); Cˢⁱᵈ = subs(Cˢʸᵐᵇ, sol)
epsilon = subs(ϵ, sol)


25.126102756390313

In [43]:
Cˢⁱᵈ 

3×3 Matrix{Polynomial{true, ComplexF64}}:
 (0.127982+0.0im)            …  (-2.62931e-5-1.85709e-6im)
 (-0.00249297-0.0809339im)      (-6.62233e-7+1.66636e-5im)
 (-2.62931e-5+1.85709e-6im)     (5.43004e-9+0.0im)

In [27]:
using DifferentialEquations

function kossak_time_evolution(ρₒ, tₘₐₓ, dt, H0, C, Fᴼᴺᴮ)
    
    # Precompute the dissipation terms
    function D!(dρ, ρ, Fᴼᴺᴮ, C)
        D = similar(ρ)
        D .= 0
        for (k, fᵢ) in enumerate(Fᴼᴺᴮ)
            for (l, fⱼ) in enumerate(Fᴼᴺᴮ)
                D .+= C[k, l] * (2 * fᵢ * ρ * fⱼ' - ρ * (fⱼ' * fᵢ) - (fⱼ' * fᵢ) * ρ)
            end
        end
        dρ .= D ./ 2
        return dρ
    end

    # Define the differential equation
    function matrix_ode!(dρ, ρ, p, t)
        dρ .= -im * (H0 * ρ - ρ * H0)
        D!(dρ, ρ, Fᴼᴺᴮ, C)
    end

    # Time span
    tspan = (0.0, tₘₐₓ)

    # Create an ODE problem
    prob = ODEProblem(matrix_ode!, ρₒ, tspan)

    # Solve the ODE using a suitable solver, e.g., Tsit5 for non-stiff problems
    sol = solve(prob, Tsit5(), dt=dt, adaptive=false)

    # Access solution
    ρ_solution = sol.u
    t_solution = sol.t
    
    return t_solution, ρ_solution
end

kossak_time_evolution (generic function with 1 method)

In [29]:
dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states

FminStates = []
FmedianStates = []
FmeanStates = []

for state in test_states # loop over initial states
    
    print(state*" ")

    start_time = time()

    tₛ, ρₛ = read_timeevolution(evol_data_file_name, state, γᵢ)
    ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
    #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
    ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
    tᵗˢᵗ = convert(Vector{Float64}, tₛ)

    #Simulated LME 
    #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵐ), [Jˢⁱᵐ])
    #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

    ρₒ = ρₛ[1]
    dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
    tᵉⁿᵈ = tᵗˢᵗ[end]

    tˢⁱᵈ, ρˢⁱᵈ  = kossak_time_evolution(ρₒ, tᵉⁿᵈ, dt, Hˢⁱᵈ, Cˢⁱᵈ, fᴷᴼᴺᴮ)
    #bˢⁱᵈ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵈ])

    #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
    F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
    #Fˢᵖⁱⁿᵇᴼˢᴼⁿ= LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)

    #Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]   

    #h5open(tests_dir*tests_data_file_name,"cw") do fid
        #γ_group = open_group(fid, γᵢ) # open gamma coupling group
        #init_state_group = create_group(γ_group, state) # create initial state group
        #init_state_group["Fidelity_SB"] = convert.(Float64, Fˢᵖⁱⁿᵇᴼˢᴼⁿ)
        #init_state_group["F"] = convert.(Float64, F)
        #init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
        #init_state_group["bloch_sid_lme"] = convert.(Float64, bˢⁱᵈ)
        #init_state_group["bloch_sim_lme"] = convert.(Float64, bˢⁱᵈ)
        #init_state_group["tr_dist_grnd"] = TrDist(ρₛ[1], ρᵍ₀)
        #init_state_group["time"] = tᵗˢᵗ
    #end
    
    FminState = minimum(F)
    FmedianState = median(F)
    FmeanState = mean(F)
    
    push!(FminStates, FminState)
    push!(FmedianStates, FmedianState)
    push!(FmeanStates, FmeanState)

end

# Calculate the mean
F_mean_value = mean(FmeanStates)

# Calculate the median
F_median_value = median(FmedianStates)

# Calculate the min
F_min_value = minimum(FminStates)

println("Median fidelity for "*γᵢ*": ", F_median_value)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.941829073543528
